# This notebook must be run with the Vizgen_2 conda environment within the Vizgen.sif singularity container

# This is for manually annotating the regions, need to save intermediate files for each brain region to speed up worflow

In [ ]:
import scanpy as sc
import matplotlib
matplotlib.use('Qt5Agg')
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import PolygonSelector
from matplotlib.path import Path
from shapely.geometry import Point, Polygon

In [ ]:
ad_viz = sc.read_h5ad('spatial_baysor_rotated.h5ad')


Region_to_annotate = 'Cerebellum'
subset_dfs = []
ad_viz.obs[Region_to_annotate] = False

In [ ]:
df = ad_viz[ad_viz.obs.batchID=='24-mo-male-4-rev2'].obs.copy()

# Create a scatter plot
fig, ax = plt.subplots()
scatter = ax.scatter(df['aligned_x'], df['aligned_y'],s=0.05)

polygon_vertices = []

# Callback function to handle the polygon drawing
def onselect(verts):
    global polygon_vertices
    polygon_vertices = verts
    # Create a Polygon object
    polygon = Polygon(polygon_vertices)
    
    # Function to check if point is inside the polygon
    def is_inside_polygon(x, y, polygon):
        point = Point(x, y)
        return polygon.contains(point)
    
    # Apply the function to the dataframe
    df['inside_polygon'] = df.apply(lambda row: is_inside_polygon(row['aligned_x'], row['aligned_y'], polygon), axis=1)
    
    # Update the scatter plot colors based on whether the points are inside the polygon
    scatter.set_color(['blue' if inside else 'red' for inside in df['inside_polygon']])
    plt.draw()

# Initialize the PolygonSelector
polygon_selector = PolygonSelector(ax, onselect)

plt.xlabel('x')
plt.ylabel('y')
plt.title('Scatter Plot with Interactive Polygon Selection')

plt.show()

In [ ]:
subset_dfs.append(df)

for new_df in subset_dfs:
    # Ensure the index of the subset dataframe matches the large dataframe
    ad_viz.obs.loc[new_df.index, Region_to_annotate] = new_df['inside_polygon']

In [ ]:
ad_viz.write_h5ad('baysor_regions_labeled.h5ad')